In [ ]:
# Importing required modules
import numpy as np  # NumPy for numerical operations
import cv2 as cv    # OpenCV for image processing
from math import sqrt # Importing sqrt function from math module
import numpy as np  # Importing NumPy again (this is redundant)
from PIL import Image  # Importing the Image class from PIL module

In [ ]:
# Defining a function named 'order_points' that takes a single argument 'pts'
def order_points(pts):
    
    # Initializing a numpy array with shape (4, 2) filled with zeros of dtype 'float32'
    rect = np.zeros((4, 2), dtype = "float32")
    
    # Calculating the sum of elements of 'pts' along the first axis (column-wise) and storing it in 's'
    s = pts.sum(axis = 1)
    
    # Assigning the point with minimum sum value of 's' to the first element of 'rect'
    rect[0] = pts[np.argmin(s)]
    
    # Assigning the point with maximum sum value of 's' to the third element of 'rect'
    rect[2] = pts[np.argmax(s)]
    
    # Computing the pairwise differences between consecutive elements of 'pts' along the second axis (row-wise) and storing it in 'diff'
    diff = np.diff(pts, axis = 1)
    
    # Assigning the point with minimum difference value of 'diff' to the second element of 'rect'
    rect[1] = pts[np.argmin(diff)]
    
    # Assigning the point with maximum difference value of 'diff' to the fourth element of 'rect'
    rect[3] = pts[np.argmax(diff)]

    # Returning the final value of 'rect' as the output of the function
    return rect

In [ ]:
def four_point_transform(image, pts):
    
    # Get the four corner points of the document
    rect = order_points(pts)
    
    # Unpack the corner points
    (tl, tr, br, bl) = rect
    
    # Compute the width of the new image
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    
    # Compute the height of the new image
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # Create an array to store the new corner points
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    
    # Compute the perspective transform matrix
    M = cv.getPerspectiveTransform(rect, dst)
    
    # Apply the perspective transform to the input image
    warped = cv.warpPerspective(image, M, (maxWidth, maxHeight))
    
    # Return the warped image
    return warped

In [ ]:
def Alignment(data, image):
    
    # Extract bounding boxes and segments data from input dictionary
    bboxes = data['boxes']
    segments = data['segments']

    # Choose a single bounding box and its corresponding segments
    bbox = bboxes[i]
    
    # Convert to integer and fix negative values
    bbox = [int(bbox) for bbox in bbox]
    bbox = [0 if i < 0 else i for i in bbox]
    
    # Convert bounding box to [x1, y1, x2, y2] format
    bbox = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
    
    # Get the segments list for this bounding box
    segment_list = segments[i][0]
    
    # Convert to integer and fix negative values
    segment_list = [int(segment_list) for segment_list in segment_list]
    segment_list = [0 if i < 0 else i for i in segment_list]
    
    # Separate x and y coordinates of the segments
    x_segement = segment_list[0::2]
    y_segement = segment_list[1::2]
    
    # Create 2D array of segments points
    segemnts_2d = np.array(list(zip(x_segement, y_segement)))
    
    # Extract the points in the segment as an array
    pts = segemnts_2d

    # Apply perspective transform to the input image using the extracted points
    warped = four_point_transform(image, pts)

    # Return the warped image
    return warped

In [ ]:
# Apply perspective transformation using Alignment function and save the result in 'warped'
warped = Alignment(data, image)